In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# List your excel file names
files = [
    "/content/drive/MyDrive/radiomics_results/3DAttention_Train_Data_All_extracted_features_OPTIMIZED_preprocessed_sequential_CT_08-25-2025_072032 (1).xlsx",
    "/content/drive/MyDrive/radiomics_results/Medsam_Train_Data_All_extracted_features_OPTIMIZED_sequential_CT_09-10-2025_190548.xlsx",
    "/content/drive/MyDrive/radiomics_results/Original_Train_Data_All_extracted_features_OPTIMIZED_sequential_CT_09-09-2025_153824.xlsx",
    "/content/drive/MyDrive/radiomics_results/Reconnet_Train_Data_All_extracted_features_OPTIMIZED_sequential_CT_09-04-2025_115208.xlsx",
    "/content/drive/MyDrive/radiomics_results/ResUnet_Train_Data_All_extracted_features_OPTIMIZED_sequential_CT_09-04-2025_120730.xlsx",
    "/content/drive/MyDrive/radiomics_results/Vnet_Train_Data_All_extracted_features_OPTIMIZED_sequential_CT_09-04-2025_225322.xlsx"
]

# Read all files
dfs = [pd.read_excel(f) for f in files]

# Step 1: Get the intersection of PatientIDs across all files
common_ids = set(dfs[0].iloc[:, 0])  # first column assumed to be PatientID
for df in dfs[1:]:
    common_ids &= set(df.iloc[:, 0])

print(f"Number of common PatientIDs across all files: {len(common_ids)}")

# Step 2: Filter each DataFrame to keep only common PatientIDs
filtered_dfs = [df[df.iloc[:, 0].isin(common_ids)] for df in dfs]

# Step 3: Save the filtered DataFrames back
for f, df in zip(files, filtered_dfs):
    out_name = f"{f}"
    df.to_excel(out_name, index=False)
    print(f"Saved: {out_name}")


Number of common PatientIDs across all files: 736
Saved: /content/drive/MyDrive/radiomics_results/3DAttention_Train_Data_All_extracted_features_OPTIMIZED_preprocessed_sequential_CT_08-25-2025_072032 (1).xlsx
Saved: /content/drive/MyDrive/radiomics_results/Medsam_Train_Data_All_extracted_features_OPTIMIZED_sequential_CT_09-10-2025_190548.xlsx
Saved: /content/drive/MyDrive/radiomics_results/Original_Train_Data_All_extracted_features_OPTIMIZED_sequential_CT_09-09-2025_153824.xlsx
Saved: /content/drive/MyDrive/radiomics_results/Reconnet_Train_Data_All_extracted_features_OPTIMIZED_sequential_CT_09-04-2025_115208.xlsx
Saved: /content/drive/MyDrive/radiomics_results/ResUnet_Train_Data_All_extracted_features_OPTIMIZED_sequential_CT_09-04-2025_120730.xlsx
Saved: /content/drive/MyDrive/radiomics_results/Vnet_Train_Data_All_extracted_features_OPTIMIZED_sequential_CT_09-04-2025_225322.xlsx


In [ ]:
import pandas as pd
from scipy.stats import shapiro

def shapiro_wilk_test(train_df, test_df, id_col="patient_id", alpha=0.05, sample_size=5000):
    """
    Apply Shapiro-Wilk test for normality on all columns (except patient_id)
    for both train and test datasets.

    Parameters:
        train_df (pd.DataFrame): Training dataset with features + patient_id
        test_df (pd.DataFrame): Test dataset with features + patient_id
        id_col (str): Column name for patient ID
        alpha (float): Significance level for hypothesis testing
        sample_size (int): Maximum sample size for Shapiro-Wilk test (scipy limitation)

    Returns:
        pd.DataFrame: Summary results for each feature and dataset
    """
    results = []

    for dataset_name, df in [("train", train_df), ("test", test_df)]:
        for col in df.columns:
            if col == id_col:
                continue  # skip patient ID column

            data = df[col].dropna().values

            # Shapiro test does not work with huge samples; limit size if needed
            if len(data) > sample_size:
                data = data[:sample_size]

            stat, p_value = shapiro(data)

            normality = "Normal" if p_value > alpha else "Not Normal"
            results.append({
                "Dataset": dataset_name,
                "Feature": col,
                "Statistic": stat,
                "p-value": p_value,
                "Normality": normality
            })

    return pd.DataFrame(results)


# Example usage:
train_df = pd.read_excel("/content/drive/MyDrive/radiomics_results/ResUnet_Train_Data_All_extracted_features_OPTIMIZED_sequential_CT_09-04-2025_120730.xlsx")
test_df = pd.read_excel("/content/drive/MyDrive/radiomics_results/ResUnet_Test_Data_All_extracted_features_OPTIMIZED_preprocessed_sequential_CT_08-25-2025_031827.xlsx")
train_df = train_df.iloc[:, 1:-1]   # removes column at index 0 (first) and last column
test_df  = test_df.iloc[:, 1:-1]
results = shapiro_wilk_test(train_df, test_df, id_col="patient_id")
results.to_excel("/content/drive/MyDrive/radiomics_results/shapiro_results/ResUnet.xlsx", index=False)


/usr/local/lib/python3.12/dist-packages/scipy/stats/_axis_nan_policy.py:579: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)
/tmp/ipython-input-1349000784.py:32: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  stat, p_value = shapiro(data)


In [ ]:
pip install pingouin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 4.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
import pingouin as pg

# ---- CONFIG ----
# ground truth train/test
ground_truth_train = "/content/drive/MyDrive/radiomics_results/Original_Train_Data_All_extracted_features_OPTIMIZED_sequential_CT_09-09-2025_153824.xlsx"
ground_truth_test  = "/content/drive/MyDrive/radiomics_results/Original_Test_data_All_extracted_features_OPTIMIZED_sequential_CT_08-29-2025_180537.xlsx"

# generated train/test (parallel files to compare with GT)
generated_train_files = [
    "/content/drive/MyDrive/radiomics_results/3DAttention_Train_Data_All_extracted_features_OPTIMIZED_preprocessed_sequential_CT_08-25-2025_072032 (1).xlsx",
    "/content/drive/MyDrive/radiomics_results/Medsam_Train_Data_All_extracted_features_OPTIMIZED_sequential_CT_09-10-2025_190548.xlsx",
    "/content/drive/MyDrive/radiomics_results/ResUnet_Train_Data_All_extracted_features_OPTIMIZED_sequential_CT_09-04-2025_120730.xlsx",
    "/content/drive/MyDrive/radiomics_results/Reconnet_Train_Data_All_extracted_features_OPTIMIZED_sequential_CT_09-04-2025_115208.xlsx",
    "/content/drive/MyDrive/radiomics_results/Vnet_Train_Data_All_extracted_features_OPTIMIZED_sequential_CT_09-04-2025_225322.xlsx"
]

generated_test_files = [
    "/content/drive/MyDrive/radiomics_results/3DAttention_Test_Data_All_extracted_features_OPTIMIZED_preprocessed_sequential_CT_08-25-2025_072032 (1).xlsx",
    "/content/drive/MyDrive/radiomics_results/Medsam_Test_Data_All_extracted_features_OPTIMIZED_sequential_CT_09-10-2025_085007.xlsx",
    "/content/drive/MyDrive/radiomics_results/ResUnet_Test_Data_All_extracted_features_OPTIMIZED_preprocessed_sequential_CT_08-25-2025_031827.xlsx",
    "/content/drive/MyDrive/radiomics_results/Reconnet_Test_Data_All_extracted_features_OPTIMIZED_preprocessed_sequential_CT_08-25-2025_032250.xlsx",
    "/content/drive/MyDrive/radiomics_results/Vnet_Test_Data_All_extracted_features_OPTIMIZED_preprocessed_sequential_CT_08-25-2025_031026 (1).xlsx"
]

# ---- Helper: compute reproducibility ----
def compute_reproducibility(gt_file, gen_file):
    gt_df = pd.read_excel(gt_file).dropna()
    gen_df = pd.read_excel(gen_file).dropna()

    patient_col = gt_df.columns[0]
    features = gt_df.columns[1:]

    # Align by PatientID
    merged = pd.merge(gt_df, gen_df, on=patient_col, suffixes=("_GT", "_GEN"))

    results = []

    for feat in features:
        gt_vals = merged[f"{feat}_GT"].values
        gen_vals = merged[f"{feat}_GEN"].values

        # Spearman
        rho, pval = spearmanr(gt_vals, gen_vals)
        # ICC(2,1)
        df_icc = pd.DataFrame({
            "Subject": np.tile(np.arange(len(gt_vals)), 2),   # repeat subjects twice
            "Rater": np.repeat([1, 2], len(gt_vals)),         # 1 = GT, 2 = Generated
            "Score": np.concatenate([gt_vals, gen_vals])      # stack values
        })


        try:
            icc_table = pg.intraclass_corr(data=df_icc,
                                           targets="Subject",
                                           raters="Rater",
                                           ratings="Score").round(3)
            icc_score = icc_table.loc[icc_table["Type"] == "ICC2", "ICC"].values[0]
        except Exception as e:
            icc_score = np.nan

        results.append([feat, icc_score, rho, pval])

    df_res = pd.DataFrame(results, columns=["Feature", "ICC(2,1)", "Spearman_rho", "Spearman_pval"])

    # Add aggregates
    agg_row = pd.DataFrame([[
        "AVERAGE",
        df_res["ICC(2,1)"].mean(),
        df_res["Spearman_rho"].mean(),
        df_res["Spearman_pval"].mean()
    ]], columns=df_res.columns)

    df_res = pd.concat([df_res, agg_row], ignore_index=True)
    return df_res


# ---- Run for all datasets ----
results = {}

# Train
for gen_file in generated_train_files:
    name = gen_file.split("/")[-1].replace(".xlsx", "")
    results[name] = compute_reproducibility(ground_truth_train, gen_file)

# Test
for gen_file in generated_test_files:
    name = gen_file.split("/")[-1].replace(".xlsx", "")
    results[name] = compute_reproducibility(ground_truth_test, gen_file)

# ---- Save Excel ----
with pd.ExcelWriter("/content/drive/MyDrive/radiomics_results/reproducibility_results.xlsx") as writer:
    for name, df in results.items():
        df.to_excel(writer, sheet_name=name[:30], index=False)

print("✅ Reproducibility results (ICC + Spearman) saved in 'reproducibility_results.xlsx'")


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_rel, wilcoxon
from statsmodels.multivariate.manova import MANOVA
from sklearn.decomposition import PCA

# ------------------
# Replace with your actual file paths
# ------------------
ground_train_path = "/content/drive/MyDrive/radiomics_results/Original_Train_Data_All_extracted_features_OPTIMIZED_sequential_CT_09-09-2025_153824.xlsx"
ground_test_path = "/content/drive/MyDrive/radiomics_results/Original_Test_data_All_extracted_features_OPTIMIZED_sequential_CT_08-29-2025_180537.xlsx"
generated_paths = { "3DAttention": ("/content/drive/MyDrive/radiomics_results/3DAttention_Train_Data_All_extracted_features_OPTIMIZED_preprocessed_sequential_CT_08-25-2025_072032 (1).xlsx", "/content/drive/MyDrive/radiomics_results/3DAttention_Test_Data_All_extracted_features_OPTIMIZED_preprocessed_sequential_CT_08-25-2025_072032 (1).xlsx"), "MedSam": ("/content/drive/MyDrive/radiomics_results/Medsam_Train_Data_All_extracted_features_OPTIMIZED_sequential_CT_09-10-2025_190548.xlsx", "/content/drive/MyDrive/radiomics_results/Medsam_Test_Data_All_extracted_features_OPTIMIZED_sequential_CT_09-10-2025_085007.xlsx"), "ResUNet": ("/content/drive/MyDrive/radiomics_results/ResUnet_Train_Data_All_extracted_features_OPTIMIZED_sequential_CT_09-04-2025_120730.xlsx", "/content/drive/MyDrive/radiomics_results/ResUnet_Test_Data_All_extracted_features_OPTIMIZED_preprocessed_sequential_CT_08-25-2025_031827.xlsx"), "ReconNet": ("/content/drive/MyDrive/radiomics_results/Reconnet_Train_Data_All_extracted_features_OPTIMIZED_sequential_CT_09-04-2025_115208.xlsx", "/content/drive/MyDrive/radiomics_results/Reconnet_Test_Data_All_extracted_features_OPTIMIZED_preprocessed_sequential_CT_08-25-2025_032250.xlsx"), "VNet": ("/content/drive/MyDrive/radiomics_results/Vnet_Train_Data_All_extracted_features_OPTIMIZED_sequential_CT_09-04-2025_225322.xlsx", "/content/drive/MyDrive/radiomics_results/Vnet_Test_Data_All_extracted_features_OPTIMIZED_preprocessed_sequential_CT_08-25-2025_031026 (1).xlsx"), }
# Load ground truth
ground_train = pd.read_excel(ground_train_path)
ground_test  = pd.read_excel(ground_test_path)

# ------------------
# Function to align datasets by PatientID
# ------------------
def align_datasets(df_gt, df_gen):
    merged = pd.merge(df_gt, df_gen, on=df_gt.columns[0], suffixes=("_gt", "_gen"))
    return merged

# ------------------
# Function to run stats on one dataset (train/test)
# ------------------
def run_stats(merged, set_name):
    feature_cols_gt  = [c for c in merged.columns if c.endswith("_gt")]
    feature_cols_gen = [c for c in merged.columns if c.endswith("_gen")]

    # ---- 1. Paired t-test ----
    ttest_rows = []
    for f_gt, f_gen in zip(feature_cols_gt, feature_cols_gen):
        x, y = merged[f_gt], merged[f_gen]
        try:
            t_stat, t_pval = ttest_rel(x, y, nan_policy="omit")
        except Exception:
            t_stat, t_pval = np.nan, np.nan
        ttest_rows.append([f_gt.replace("_gt", ""), t_stat, t_pval])
    df_ttest = pd.DataFrame(ttest_rows, columns=["Feature", "T_statistic", "P_value"])

    # ---- 2. Wilcoxon ----
    wilcoxon_rows = []
    for f_gt, f_gen in zip(feature_cols_gt, feature_cols_gen):
        x, y = merged[f_gt], merged[f_gen]
        try:
            w_stat, w_pval = wilcoxon(x, y)
        except Exception:
            w_stat, w_pval = np.nan, np.nan
        wilcoxon_rows.append([f_gt.replace("_gt", ""), w_stat, w_pval])
    df_wilcoxon = pd.DataFrame(wilcoxon_rows, columns=["Feature", "W_statistic", "P_value"])

    # ---- 3. MANOVA with PCA ----
    try:
        records = []
        for f_gt, f_gen in zip(feature_cols_gt, feature_cols_gen):
            feature_name = f_gt.replace("_gt","")
            for pid, gt_val, gen_val in zip(merged.iloc[:,0], merged[f_gt], merged[f_gen]):
                records.append({"PatientID": pid, "Group": "GT", feature_name: gt_val})
                records.append({"PatientID": pid, "Group": "GEN", feature_name: gen_val})
        manova_df = pd.DataFrame(records)

        manova_df = manova_df.pivot_table(index=["PatientID","Group"],
                                          values=[c for c in manova_df.columns if c not in ["PatientID","Group"]],
                                          aggfunc="first").reset_index()

        feature_cols = [c for c in manova_df.columns if c not in ["PatientID","Group"]]
        X = manova_df[feature_cols].fillna(0).values

        n_components = min(20, X.shape[1])
        pca = PCA(n_components=n_components)
        X_pca = pca.fit_transform(X)

        pca_cols = [f"PC{i+1}" for i in range(X_pca.shape[1])]
        X_df = pd.DataFrame(X_pca, columns=pca_cols)
        X_df["Group"] = manova_df["Group"].values

        formula = f"{' + '.join(pca_cols)} ~ Group"
        maov = MANOVA.from_formula(formula, data=X_df)
        manova_summary = maov.mv_test()

        stat_table = manova_summary.results["Group"]["stat"].copy()
        stat_table.index.name = "Test"
        df_manova = stat_table.reset_index()

        pvals = df_manova["Pr > F"].astype(float)
        decision = "Reject H0 (Significant difference)" if (pvals < 0.05).any() else "Fail to reject H0 (No significant difference)"
        summary_row = pd.DataFrame([["Overall Decision", "-", "-", "-", "-", decision]],
                                   columns=df_manova.columns)
        df_manova = pd.concat([df_manova, summary_row], ignore_index=True)

    except Exception as e:
        df_manova = pd.DataFrame({"Error": [str(e)]})

    return {f"Paired_ttest_{set_name}": df_ttest,
            f"Wilcoxon_{set_name}": df_wilcoxon,
            f"MANOVA_{set_name}": df_manova}


# ------------------
# Run for each generator (train & test separately)
# ------------------
for gen_name, (gen_train_path, gen_test_path) in generated_paths.items():
    gen_train = pd.read_excel(gen_train_path)
    gen_test  = pd.read_excel(gen_test_path)

    merged_train = align_datasets(ground_train, gen_train)
    merged_test  = align_datasets(ground_test, gen_test)

    stats_train = run_stats(merged_train, "Train")
    stats_test  = run_stats(merged_test, "Test")

    out_path = f"/content/drive/MyDrive/radiomics_results/reproducibility_stats_{gen_name}.xlsx"
    with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
        for sheet_name, df in {**stats_train, **stats_test}.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"✅ Saved results for {gen_name} -> {out_path}")


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import os
import numpy as np

# ================= Configurable Parameters =================
folder_path = "/content/drive/MyDrive/radiomics_results/eval_stat"
colors = {"whole": "#1D0FDB", "crop": "#DB0F38"}
figsize = (16, 8)
ymin, ymax, ystep = 0.0, 45, 4
line_thickness = 1.5
line_style = "--"
line_color = "#383535"
# ===========================================================

# Load and merge all Excel files
all_dfs = []
for file in glob.glob(os.path.join(folder_path, "*.xlsx")):
    model_name = os.path.splitext(os.path.basename(file))[0]
    if any(m in model_name for m in ["3DAttention","MedSam","ResUNet","ReconNet","VNet"]):
        df = pd.read_excel(file)
        if "whole" in model_name.lower():
            category = "whole"
        elif "crop" in model_name.lower():
            category = "crop"
        else:
            raise ValueError(f"Category not found in {file}")

        df["Model"] = next(m for m in ["3DAttention","MedSam","ResUNet","ReconNet","VNet"] if m in model_name)
        df["Category"] = category
        all_dfs.append(df)

data = pd.concat(all_dfs, ignore_index=True)
data.rename(columns=lambda x: x.strip(), inplace=True)  # strip spaces
# ================= Plotting =================
sns.set(style="whitegrid", font_scale=1)
plt.figure(figsize=figsize)
ax = plt.gca()

# --- Boxplots ---
sns.boxplot(
    data=data,
    x="Model",
    y="Hausdorff Distance",
    hue="Category",
    palette=colors,
    width=0.4,
    dodge=True,
    ax=ax
)

# --- Scatter points ---
sns.stripplot(
    data=data,
    x="Model",
    y="Hausdorff Distance",
    hue="Category",
    dodge=True,
    palette=colors,
    alpha=0.7,
    linewidth=0.5,
    edgecolor="black",
    ax=ax
)

# Remove duplicate legends (since both box & strip added)
# handles, labels = ax.get_legend_handles_labels()
# ax.legend(handles[:2], labels[:2], title="Category")

# --- Extract median line positions ---
median_positions = {}
for line in ax.lines:
    if line.get_linestyle() == '-' and np.isclose(line.get_ydata()[0], line.get_ydata()[1]):
        # This is a median line
        x_coords = line.get_xdata()
        y_coords = line.get_ydata()
        median_x = np.mean(x_coords)
        median_y = y_coords[0]

        # Find nearest tick (model index)
        model_idx = int(round(median_x))
        if 0 <= model_idx < len(ax.get_xticks()):
            model_name = ax.get_xticklabels()[model_idx].get_text()

            # Decide which category: left (whole) or right (crop)
            if median_x < model_idx:
                cat = "whole"
            else:
                cat = "crop"

            median_positions.setdefault(model_name, {})[cat] = (median_x, median_y)

# --- Draw connecting lines ---
for model, coords in median_positions.items():
    if "whole" in coords and "crop" in coords:
        x_vals = [coords["whole"][0], coords["crop"][0]]
        y_vals = [coords["whole"][1], coords["crop"][1]]
        ax.plot(
            x_vals, y_vals,
            linestyle=line_style,
            linewidth=line_thickness,
            color=line_color
        )

# --- Customization ---
ax.set_ylim(ymin, ymax)
ax.set_yticks(np.arange(ymin, ymax + ystep, ystep))
ax.legend_.remove()
for spine in ax.spines.values():
    spine.set_visible(True)

plt.tight_layout()

plt.savefig("/content/drive/MyDrive/val_Hausdorff.png", dpi=400, bbox_inches="tight")
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import os
import numpy as np

# ================= Configurable Parameters =================
folder_path = "/content/drive/MyDrive/radiomics_results/eval_stat"
colors = {"whole": "#1D0FDB", "crop": "#DB0F38"}
figsize = (16, 8)
ymin, ymax, ystep = 0.0, 1.0, 0.1
line_thickness = 1.5
line_style = "--"
line_color = "#383535"
target_col = "Hausdorff Distance"
# ===========================================================

# Load and merge all Excel files
all_dfs = []
for file in glob.glob(os.path.join(folder_path, "*.xlsx")):
    model_name = os.path.splitext(os.path.basename(file))[0]

    if any(m in model_name for m in ["3DAttention","MedSam","ResUNet","ReconNet","VNet"]):
        df = pd.read_excel(file)

        # --- Normalize column names ---
        df.columns = df.columns.str.strip()

        if target_col not in df.columns:
            print(f"⚠️ Skipping {file}, no column '{target_col}' found.")
            continue

        # --- Identify category ---
        if "whole" in model_name.lower():
            category = "whole"
        elif "crop" in model_name.lower():
            category = "crop"
        else:
            raise ValueError(f"Category not found in {file}")

        # --- Add metadata ---
        df["Model"] = next(m for m in ["3DAttention","MedSam","ResUNet","ReconNet","VNet"] if m in model_name)
        df["Category"] = category

        # --- Ensure numeric column ---
        df[target_col] = pd.to_numeric(df[target_col], errors="coerce")

        all_dfs.append(df)

# Combine data
if not all_dfs:
    raise ValueError("❌ No valid data loaded. Check filenames and column names.")
data = pd.concat(all_dfs, ignore_index=True)

print("✅ Data loaded:", data.shape)
print(data[[target_col, "Model", "Category"]].head())

# ================= Plotting =================
sns.set(style="whitegrid", font_scale=1.2)
plt.figure(figsize=figsize)
ax = plt.gca()

# --- Boxplots ---
sns.boxplot(
    data=data,
    x="Model",
    y=target_col,
    hue="Category",
    palette=colors,
    width=0.8,
    dodge=True,
    ax=ax
)

# --- Scatter points ---
sns.stripplot(
    data=data,
    x="Model",
    y=target_col,
    hue="Category",
    dodge=True,
    palette=colors,
    alpha=0.7,
    linewidth=0.5,
    edgecolor="black",
    ax=ax
)

# Remove duplicate legends (since both box & strip added)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:2], labels[:2], title="Category")

# --- Customization ---
ax.set_ylim(ymin, ymax)
ax.set_yticks(np.arange(ymin, ymax + ystep, ystep))
ax.legend_.remove()
for spine in ax.spines.values():
    spine.set_visible(True)

plt.tight_layout()
plt.savefig("/content/drive/MyDrive/val_Hausdorff.png", dpi=400, bbox_inches="tight")
plt.show()
